# Naive Bayes

## 1. Pros
works with small amount of data. 
Handles multiple classes

## 2. Cons
sensitive to how the input data is prepared

## 3. Works with
nominal values

## 4. Bayesian decision theory
choose the class with higher probability
Probility that point (x, y) belongs to class c_i
$$ p(c_i | x,y) = {{p(x,y|c_i)p(c_i)} \over {p(x,y)}}$$

token: any combination of words

## 5. filter for online board
### 1. make word vectors from text

In [1]:
import numpy as np

def loadDataSet():
    postingList = [["my", "dog", "has", "flea", \
                   "problems", "help", "please"],
                  ["maybe", "not", "take", "him", \
                  "to", "dog", "park", "stupid"],
                  ["my", "dalmation", "is", "so", "cute", \
                  "I", "love", "him"],
                  ["stop", "posting", "stupid", "worthless", "garbage"],
                  ["mr", "licks", "ate", "my", "steak", "how", \
                  "to", "stop", "him"],
                  ["quit", "buying", "worthless", "dog", "food", "stupid"]]
    classVec = [0, 1, 0, 1, 0, 1] # 1 is abusive, 0 not
    return postingList, classVec

def createVocabList(dataSet):
    vocabSet = set([])
    for document in dataSet:
        vocabSet = vocabSet | set(document)
    return list(vocabSet)

def setOfWords2Vec(vocabList, inputSet):
    returnVec = np.array([0]*len(vocabList))
    for word in inputSet:
        if word in vocabList:
            returnVec[vocabList.index(word)] = 1
        else: print ("the word: %s is not in my Vocabulary!" % word)
    return returnVec

In [2]:
listOPosts, listClasses = loadDataSet()
myVocabList = createVocabList(listOPosts)

In [3]:
myVocabList.index("cute")

5

In [4]:
setOfWords2Vec(myVocabList, listOPosts[0])

array([0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 1, 0, 1, 1, 0])

### 2. calculate probability
$$ p(c_1 | w) = {{p(w|c_1)p(c_1)} \over {p(w)}}$$
$$w = (w_1, w_2, w_3, ... w_n)$$
c_1: this document is abusive

w_1: the occurance of word i

In [5]:
def trainNB0(trainMatrix, trainCategory):
    numTrainDocs = len(trainMatrix)
    numWords = len(trainMatrix[0])
    pAbusive = sum(trainCategory)/float(numTrainDocs)
    p0Num = np.ones(numWords); p1Num = np.ones(numWords)
    p0Denom = 2; p1Denom = 2
    for i in range(numTrainDocs):
        if trainCategory[i] == 1:
            p1Num += trainMatrix[i]
            p1Denom += sum(trainMatrix[i])
        else:
            p0Num += trainMatrix[i]
            p0Denom += sum(trainMatrix[i])
    p1Vect = np.log(p1Num/p1Denom) # number of word occurance/total 
                           # number of all words in vocab
    p0Vect = np.log(p0Num/p0Denom)
    return p0Vect, p1Vect, pAbusive

In [6]:
trainMat = np.zeros((0,len(myVocabList)))
for postinDoc in listOPosts:
    trainMat = np.append(trainMat, setOfWords2Vec(myVocabList, postinDoc)[np.newaxis,:], axis=0)

In [7]:
p0V, p1V, pAb = trainNB0(trainMat, listClasses)

In [8]:
pAb

0.5

In [9]:
p0V[18]

-3.258096538021482

In [10]:
p1V[18]

-2.3513752571634776

### 3. flaws

1. we multiply a lot possibility together to get $ p(w | c_1)$, if any of them is zero, the result is zero. To tackle this, we initialize all occurance counts to 1, and all denominators to 2.
2. Underflow: doing too many multiplication of small numbers in python, eventually it rounds off to 0. Solution: **Take the natural log of this product**, since $ln(a*b) = ln(a) + ln(b)$, natural log avoids underflow.

### 4. classfier

In [11]:
def classifyNB(vec2Classify, p0Vec, p1Vec, pClass1):
    p1 = np.sum(vec2Classify * p1Vec) + np.log(pClass1)
    p0 = np.sum(vec2Classify * p0Vec) + np.log(1 - pClass1)
    if p1 > p0:
        return 1
    else:
        return 0
def testingNB(): #wrapper
    listOPosts, listClasses = loadDataSet()
    myVocabList = createVocabList(listOPosts)
    trainMat = np.zeros((0,len(myVocabList)))
    for postinDoc in listOPosts:
        trainMat = np.append(trainMat, setOfWords2Vec(myVocabList, postinDoc)[np.newaxis,:], axis=0)
    p0V, p1V, pAb = trainNB0(trainMat, np.array(listClasses))
    testEntry = ["love", "my", "dalmation"]
    thisDoc = np.array(setOfWords2Vec(myVocabList, testEntry))
    print(testEntry, "classified as: ", classifyNB(thisDoc, p0V, p1V, pAb))
    testEntry = ["stupid", "garbage"]
    thisDoc = np.array(setOfWords2Vec(myVocabList, testEntry))
    print(testEntry, "classified as: ", classifyNB(thisDoc, p0V, p1V, pAb))

In [12]:
testingNB()

['love', 'my', 'dalmation'] classified as:  0
['stupid', 'garbage'] classified as:  1


### 5. improve, bag-of-words model
every time we encounter a word, we increase word vector instead of setting it to 1

In [13]:
def bagOfWords2VecMN(vocabList, inputSet):
    returnVec = np.zeros(len(vocabList))
    for word in inputSet:
        if word in vocabList:
            returnVec[vocabList.index(word)] += 1
        else: print ("the word: %s is not in my Vocabulary!" % word)
    return returnVec

## 6. classifying spam email
1. Collect data
2. Prepare: text -> token vector
3. Analyze: doble-check tokens
4. Train: trainNB0()
5. Test: classifyNB()
6. Use

In [14]:
mySent = "This book is the best book on Python or M.L. I have laid eyes upon."
mySent.split()

['This',
 'book',
 'is',
 'the',
 'best',
 'book',
 'on',
 'Python',
 'or',
 'M.L.',
 'I',
 'have',
 'laid',
 'eyes',
 'upon.']

that is good, but note the punctuation in *"upon."*

In [15]:
# alternative: use regex
import re
regEx = re.compile('\W+') # W: any non-word
mySent = "This book is the best book on Python or M.L. I have laid eyes upon."
listOfTokens = regEx.split(mySent)
listOfTokens

['This',
 'book',
 'is',
 'the',
 'best',
 'book',
 'on',
 'Python',
 'or',
 'M',
 'L',
 'I',
 'have',
 'laid',
 'eyes',
 'upon',
 '']

looks good to me, but there is an empty string at index -1, and the first letter is capitalized

In [16]:
[tok.lower() for tok in listOfTokens if len(tok) > 0]

['this',
 'book',
 'is',
 'the',
 'best',
 'book',
 'on',
 'python',
 'or',
 'm',
 'l',
 'i',
 'have',
 'laid',
 'eyes',
 'upon']

In [17]:
def textParse(bigString):
    listOfTokens = re.split('\W+', bigString)
    regex = re.compile('[0-9]+')  # filter out words with numbers
    return [tok.lower() for tok in listOfTokens if (len(tok) > 2 and not regex.search(tok))]    

In [18]:
import os
import io

def ret_data(): 
    ham_list = []
    spam_list = []

    for filename in os.listdir("./data/email/ham/"):
        f = open("./data/email/ham/" + filename, mode="r", encoding="latin-1")
        ham_list.append(f.read())

    for filename in os.listdir("./data/email/spam/"):
        f = open("./data/email/spam/" + filename, mode="r", encoding="latin-1")
        spam_list.append(f.read())
    return ham_list, spam_list

In [19]:
ham_list, spam_list = ret_data()

In [20]:
 def createDataSet(ham_list, spam_list):
    train_ham_list = ham_list[:200]
    test_ham_list = ham_list[200:]
    train_spam_list = spam_list[:50]
    test_spam_list = spam_list[50:]
    listOfPost = []
    listOfClass = []
    for post in train_ham_list:
        listOfPost.append(textParse(post))
        listOfClass.append(0) # 0 is ham, 1 is spam
    for post in train_spam_list:
        listOfPost.append(textParse(post))
        listOfClass.append(1)
    return listOfPost, listOfClass, 

In [21]:
posts, classes = createDataSet(ham_list, spam_list)
vocabList = createVocabList(posts)

In [22]:
trainMat = np.zeros((0,len(vocabList)))
for postinDoc in posts:
    trainMat = np.append(trainMat, bagOfWords2VecMN(vocabList, postinDoc)[np.newaxis,:], axis=0)

In [23]:
# def testingNB(): #wrapper
#     listOPosts, listClasses = loadDataSet()
#     myVocabList = createVocabList(listOPosts)
#     trainMat = np.zeros((0,len(myVocabList)))
#     for postinDoc in listOPosts:
#         trainMat = np.append(trainMat, setOfWords2Vec(myVocabList, postinDoc)[np.newaxis,:], axis=0)
#     p0V, p1V, pAb = trainNB0(trainMat, np.array(listClasses))
#     testEntry = ["love", "my", "dalmation"]
#     thisDoc = np.array(setOfWords2Vec(myVocabList, testEntry))
#     print(testEntry, "classified as: ", classifyNB(thisDoc, p0V, p1V, pAb))
#     testEntry = ["stupid", "garbage"]
#     thisDoc = np.array(setOfWords2Vec(myVocabList, testEntry))
#     print(testEntry, "classified as: ", classifyNB(thisDoc, p0V, p1V, pAb))

In [24]:
p0V, p1V, pAb = trainNB0(trainMat, np.array(classes))

In [37]:
count = []
for i in range(len(spam_list)//5):
    testEntry = textParse(spam_list[50 + i])
    thisDoc = np.array(bagOfWords2VecMN(vocabList, testEntry))
    count.append(classifyNB(thisDoc, p0V, p1V, pAb))

the word: zahrae is not in my Vocabulary!
the word: qves is not in my Vocabulary!
the word: zahrae is not in my Vocabulary!
the word: qves is not in my Vocabulary!
the word: qves is not in my Vocabulary!
the word: qves is not in my Vocabulary!
the word: zahrae is not in my Vocabulary!
the word: qves is not in my Vocabulary!
the word: hottest is not in my Vocabulary!
the word: freeyankeedom is not in my Vocabulary!
the word: hottest is not in my Vocabulary!
the word: adclick is not in my Vocabulary!
the word: lbs is not in my Vocabulary!
the word: patches is not in my Vocabulary!
the word: adclick is not in my Vocabulary!
the word: adclick is not in my Vocabulary!
the word: qves is not in my Vocabulary!
the word: trim is not in my Vocabulary!
the word: hottest is not in my Vocabulary!
the word: adclick is not in my Vocabulary!
the word: _blank is not in my Vocabulary!
the word: lbs is not in my Vocabulary!
the word: adclick is not in my Vocabulary!
the word: _blank is not in my Vocabula

the word: nbc is not in my Vocabulary!
the word: nbc is not in my Vocabulary!
the word: pru is not in my Vocabulary!
the word: nbc is not in my Vocabulary!
the word: nbc is not in my Vocabulary!
the word: qqejiga is not in my Vocabulary!
the word: prulife is not in my Vocabulary!
the word: protector is not in my Vocabulary!
the word: prulife is not in my Vocabulary!
the word: closed is not in my Vocabulary!
the word: premiums is not in my Vocabulary!
the word: endow is not in my Vocabulary!
the word: premiums is not in my Vocabulary!
the word: prudential is not in my Vocabulary!
the word: prulife is not in my Vocabulary!
the word: protectorsm is not in my Vocabulary!
the word: premiums is not in my Vocabulary!
the word: prulife is not in my Vocabulary!
the word: plussm is not in my Vocabulary!
the word: premiums is not in my Vocabulary!
the word: endow is not in my Vocabulary!
the word: prudential is not in my Vocabulary!
the word: _____ is not in my Vocabulary!
the word: commissionabl

the word: ___________________________________________________________________ is not in my Vocabulary!
the word: _______________________________________________________________ is not in my Vocabulary!
the word: ________________________________________________________________ is not in my Vocabulary!
the word: _____________________________________________________________________ is not in my Vocabulary!
the word: ___________________________________________________________________ is not in my Vocabulary!
the word: ___________________________________________________________________ is not in my Vocabulary!
the word: __________________________________________________________________ is not in my Vocabulary!
the word: ___________________________________________________________________________ is not in my Vocabulary!
the word: ___________________________________________________________________________ is not in my Vocabulary!
the word: _____________________________________________________

the word: lbfbdhvunx is not in my Vocabulary!
the word: teeth is not in my Vocabulary!
the word: whitening is not in my Vocabulary!
the word: ovcnd is not in my Vocabulary!
the word: whiter is not in my Vocabulary!
the word: teeth is not in my Vocabulary!
the word: smile is not in my Vocabulary!
the word: dental is not in my Vocabulary!
the word: teeth is not in my Vocabulary!
the word: whitening is not in my Vocabulary!
the word: dentist is not in my Vocabulary!
the word: professionally is not in my Vocabulary!
the word: whiten is not in my Vocabulary!
the word: teeth is not in my Vocabulary!
the word: whitening is not in my Vocabulary!
the word: dentist is not in my Vocabulary!
the word: fraction is not in my Vocabulary!
the word: dental is not in my Vocabulary!
the word: cxbiz is not in my Vocabulary!
the word: cxbiz is not in my Vocabulary!
the word: joem is not in my Vocabulary!
the word: efilm is not in my Vocabulary!
the word: haizhong is not in my Vocabulary!
the word: efile is

the word: mortg is not in my Vocabulary!
the word: click_here is not in my Vocabulary!
the word: mortg is not in my Vocabulary!
the word: chalres is not in my Vocabulary!
the word: gillette is not in my Vocabulary!
the word: mortcorp is not in my Vocabulary!
the word: olhfilghxvuyfkfqbviqgacyenuuufpyswymlajxucqhpojagplujpmovuiaphwlqkkstgipemliwuugqptjukjklbjlgxqmuogtiwhhdlhjhkbucvbyqvgfp is not in my Vocabulary!
the word: __________________________________________________________________ is not in my Vocabulary!
the word: bestoffersonthenet is not in my Vocabulary!
the word: bestoffersonthenet is not in my Vocabulary!
the word: bestoffersonthenet is not in my Vocabulary!
the word: bestoffersonthenet is not in my Vocabulary!
the word: bestoffersonthenet is not in my Vocabulary!
the word: bestoffersonthenet is not in my Vocabulary!
the word: hbncikzddu is not in my Vocabulary!
the word: bestoffersonthenet is not in my Vocabulary!
the word: bestoffersonthenet is not in my Vocabulary!
the 

...

the word: nosend is not in my Vocabulary!
the word: bottom_bg is not in my Vocabulary!
the word: listserv is not in my Vocabulary!
the word: sitehoster is not in my Vocabulary!
the word: listserv is not in my Vocabulary!
the word: yourhostbiz is not in my Vocabulary!
the word: sitehoster is not in my Vocabulary!
the word: nosend is not in my Vocabulary!
the word: tml is not in my Vocabulary!
the word: runbox is not in my Vocabulary!
the word: runbox is not in my Vocabulary!
the word: monat is not in my Vocabulary!
the word: svr is not in my Vocabulary!
the word: klafu is not in my Vocabulary!
the word: surfer is not in my Vocabulary!
the word: mxs is not in my Vocabulary!
the word: monat is not in my Vocabulary!
the word: svr is not in my Vocabulary!
the word: aibo is not in my Vocabulary!
the word: runbox is not in my Vocabulary!
the word: grace is not in my Vocabulary!
the word: quentin is not in my Vocabulary!
the word: runbox is not in my Vocabulary!
the word: _____________________

the word: conditioning is not in my Vocabulary!
the word: headlamp is not in my Vocabulary!
the word: wash is not in my Vocabulary!
the word: restraints is not in my Vocabulary!
the word: tinted is not in my Vocabulary!
the word: trim is not in my Vocabulary!
the word: msonormal is not in my Vocabulary!
the word: padding is not in my Vocabulary!
the word: msonormal is not in my Vocabulary!
the word: padding is not in my Vocabulary!
the word: plates is not in my Vocabulary!
the word: pcp is not in my Vocabulary!
the word: pch is not in my Vocabulary!
the word: sesystems is not in my Vocabulary!
the word: sesystems is not in my Vocabulary!
the word: sesystems is not in my Vocabulary!
the word: sesystems is not in my Vocabulary!
the word: dyporn is not in my Vocabulary!
the word: dyporn is not in my Vocabulary!
the word: midsouth is not in my Vocabulary!
the word: dyporn is not in my Vocabulary!
the word: dyporn is not in my Vocabulary!
the word: adults is not in my Vocabulary!
the word: 

the word: advises is not in my Vocabulary!
the word: earning is not in my Vocabulary!
the word: sheet is not in my Vocabulary!
the word: sheet is not in my Vocabulary!
the word: stamps is not in my Vocabulary!
the word: envelopes is not in my Vocabulary!
the word: ________________________________________________ is not in my Vocabulary!
the word: speaker is not in my Vocabulary!
the word: derrick is not in my Vocabulary!
the word: nguyen is not in my Vocabulary!
the word: whitmore is not in my Vocabulary!
the word: monte is not in my Vocabulary!
the word: ________________________________________________ is not in my Vocabulary!
the word: fai is not in my Vocabulary!
the word: gpo is not in my Vocabulary!
the word: macau is not in my Vocabulary!
the word: hong is not in my Vocabulary!
the word: kong is not in my Vocabulary!
the word: ________________________________________________ is not in my Vocabulary!
the word: ________________________________________________ is not in my Vocabular

the word: offersrus is not in my Vocabulary!
the word: signup is not in my Vocabulary!
the word: offersrus is not in my Vocabulary!
the word: offersrus is not in my Vocabulary!
the word: offersrus is not in my Vocabulary!
the word: bullet is not in my Vocabulary!
the word: internets is not in my Vocabulary!
the word: offersrus is not in my Vocabulary!
the word: bullet is not in my Vocabulary!
the word: reviews is not in my Vocabulary!
the word: topics is not in my Vocabulary!
the word: offersrus is not in my Vocabulary!
the word: bullet is not in my Vocabulary!
the word: offersrus is not in my Vocabulary!
the word: bullet is not in my Vocabulary!
the word: supplier is not in my Vocabulary!
the word: resource is not in my Vocabulary!
the word: offersrus is not in my Vocabulary!
the word: ad_mail_mega is not in my Vocabulary!
the word: reach_the_masses is not in my Vocabulary!
the word: offersrus is not in my Vocabulary!
the word: mailed_by_ is not in my Vocabulary!
the word: offersrus i

the word: endeavor is not in my Vocabulary!
the word: seventeen is not in my Vocabulary!
the word: proposition is not in my Vocabulary!
the word: rudiments is not in my Vocabulary!
the word: endeavor is not in my Vocabulary!
the word: obliged is not in my Vocabulary!
the word: barrister is not in my Vocabulary!
the word: kola is not in my Vocabulary!
the word: owolabi is not in my Vocabulary!
the word: owolabi is not in my Vocabulary!
the word: __________________________________________________________________ is not in my Vocabulary!
the word: stores is not in my Vocabulary!
the word: shopping is not in my Vocabulary!
the word: convenience is not in my Vocabulary!
the word: shopnow is not in my Vocabulary!


In [39]:
count.count(1)

92